## setup envirionment

In [5]:
import os
import sys 

from main import *
sys.path.insert(0,"../common/")
from utils import *

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Config

In [6]:

name = 'fp_exp3'
model_savename = 'longformer'

model_name = '../input/pt-longformer-base' # https://www.kaggle.com/kishalmandal/pt-longformer-base
base_dir = '../'
data_dir = '../data/feedback-prize-2021/'
pre_data_dir = '../preprocessed/'
model_dir = '.'
output_dir = '.'

n_epoch = 1 # not to exceed runtime limits on Kaggle
n_fold = 5
verbose_steps = 500
random_seed = 42
max_length = 1024
train_batch_size = 4
valid_batch_size = 4
lr = 5e-5
num_labels = 15
label_subtokens = True
output_hidden_states = True
hidden_dropout_prob = 0.1
layer_norm_eps = 1e-7
add_pooling_layer = False
verbose_steps = 500

IGNORE_INDEX = -100
NON_LABEL = -1
OUTPUT_LABELS = ['0', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
                 'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']
LABELS_TO_IDS = {v:k for k,v in enumerate(OUTPUT_LABELS)}
IDS_TO_LABELS = {k:v for k,v in enumerate(OUTPUT_LABELS)}

MIN_THRESH = {
    "I-Lead": 9,
    "I-Position": 5,
    "I-Evidence": 14,
    "I-Claim": 3,
    "I-Concluding Statement": 11,
    "I-Counterclaim": 6,
    "I-Rebuttal": 4,
}

PROB_THRESH = {
    "I-Lead": 0.7,
    "I-Position": 0.55,
    "I-Evidence": 0.65,
    "I-Claim": 0.55,
    "I-Concluding Statement": 0.7,
    "I-Counterclaim": 0.5,
    "I-Rebuttal": 0.55,
}


In [7]:
# general
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import gc
from collections import defaultdict
# nlp
from sklearn.feature_extraction.text import CountVectorizer
import torch
import torch.nn as nn
from transformers import LongformerConfig, LongformerModel, LongformerTokenizerFast
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

In [29]:
def get_preds_folds(df, dl, criterion, valid_flg=False):
    for i_fold in range(2):
        model_filename = os.path.join(config['model_dir'], f"{config['model_savename']}_{i_fold}.bin")
        print(f"{model_filename} inference")
        model = FeedbackModel()
        model = model.to(device)
        model.load_state_dict(torch.load(model_filename))
        logits, valid_loss, valid_acc = inference(model, dl, criterion, valid_flg)
        if i_fold == 0:
            avg_pred_logits = logits
        else:
            avg_pred_logits += logits
    avg_pred_logits /= config['n_fold']
    all_preds, all_preds_prob = preds_class_prob(avg_pred_logits, dl)
    df_pred = post_process_pred(df, all_preds, all_preds_prob)
    return df_pred

In [30]:
train_flg = False
tokenizer = LongformerTokenizerFast.from_pretrained(config['model_name'], add_prefix_space = True)

df_submission_prep = preprocess()


100%|██████████| 5/5 [00:00<00:00, 744.73it/s]

Completed tokenizing texts.


In [31]:
subm_idlist = df_submission_prep['id'].unique().tolist()
ds_submission_prep = FeedbackPrizeDataset(df_submission_prep, tokenizer, config['max_length'], True)
dl_submission_prep = DataLoader(ds_submission_prep, batch_size=config['valid_batch_size'], shuffle=False, num_workers=2, pin_memory=True)


In [33]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.cuda.set_device(7) 
else:
    device = torch.device('cpu')
    
import os

criterion = ''
get_preds_folds(df_submission_prep, dl_submission_prep, criterion, valid_flg=False)

../model/fp_exp3/longformer_0.bin inference


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA out of memory. Tried to allocate 148.00 MiB (GPU 0; 10.92 GiB total capacity; 160.11 MiB already allocated; 22.38 MiB free; 164.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF